In [1]:
# conda install -c conda-forge shap
# pip install xgboost
# pip install lightgbm

# 순서
1. 파일읽기
2. 모델링 및 모델 분석
    1. Randomforest
    1. GBM
    1. XGBoost
3. 결론 

# 파일 읽기  Import Data

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
import shap
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score



In [25]:
#Reading the train and test files
train_prod_df = pd.read_csv('data\\train.csv') # 학습데이터
test_prod_df = pd.read_csv('data\\test_empty.csv') # 테스트데이터 (결과값 비어있음)

In [26]:
train_prod_df.head()

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,...,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,1,open,choongnam,73,20071228,175.0,nursing_hospital,4.217530e+09,0.0,3.961135e+09,...,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,same
1,3,open,gyeongnam,32,19970401,410.0,general_hospital,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,801.0,813.0,same
2,4,open,gyeonggi,89,20161228,468.0,nursing_hospital,1.004522e+09,515483669.0,4.472197e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,same
3,7,open,incheon,141,20000814,353.0,general_hospital,7.250734e+10,0.0,7.067740e+10,...,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,same
4,9,open,gyeongnam,32,20050901,196.0,general_hospital,4.904354e+10,0.0,4.765605e+10,...,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,same


In [27]:
test_prod_df.head()

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,...,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,2,NaN,incheon,139,19981125.0,300.0,general_hospital,6.682486e+10,0.000000e+00,6.565709e+10,...,5.540643e+10,5.068443e+10,3.714334e+10,4.720000e+09,4.690000e+09,1.608540e+10,8.944587e+09,693,693,same
1,5,NaN,jeju,149,20160309.0,44.0,hospital,3.495758e+10,0.000000e+00,3.259270e+10,...,6.730838e+10,4.209828e+10,2.420000e+10,2.521009e+10,1.830000e+10,3.789135e+09,0.000000e+00,379,371,same
2,6,NaN,jeonnam,103,19890427.0,276.0,general_hospital,2.326031e+10,2.542571e+09,2.308749e+10,...,0.000000e+00,2.777589e+10,2.182278e+10,0.000000e+00,0.000000e+00,0.000000e+00,1.638540e+10,NaN,NaN,NaN
3,8,NaN,busan,71,20100226.0,363.0,general_hospital,0.000000e+00,0.000000e+00,0.000000e+00,...,1.211517e+10,9.556237e+09,4.251867e+09,2.558931e+09,0.000000e+00,3.914284e+10,0.000000e+00,760,760,same
4,10,NaN,jeonbuk,26,20040604.0,213.0,general_hospital,5.037025e+10,0.000000e+00,4.855803e+10,...,4.395973e+10,7.535567e+09,3.298427e+09,3.642417e+10,2.134712e+10,2.574488e+10,1.507269e+10,437,385,same


## 데이터 전처리

#### 형변환

In [6]:
#Removing the comma in the employee1 and 2 columns in the test dataset and replace it with empty space and convert it to float format.
test_prod_df.employee1 = test_prod_df.employee1.astype('str').str.replace(",", "").astype('float')
test_prod_df.employee2 = test_prod_df.employee2.astype('str').str.replace(",", "").astype('float')

In [7]:
#Converting the employee1 and 2 column as float in the train set as done for the test dataset
train_prod_df.employee1 = train_prod_df.employee1.astype('float')
train_prod_df.employee2 = train_prod_df.employee2.astype('float')
train_prod_df.OC= train_prod_df.OC.astype('str').str.replace(" ","")

In [8]:
#Combining the train and test dataset
train_test_prod = train_prod_df.append(test_prod_df)

#### 레이블 인코딩

In [10]:
#Get the object and numeric columns seperately 
factor_columns = train_test_prod.select_dtypes(include = ['object']).columns
numeric_columns = train_test_prod.columns.difference(factor_columns)

In [11]:
#After analysis realized that the bed counts of these two hospitals may have had wrong entries.
#Filling up the empty instkind and bedCount for hospital id 430 and 413
train_test_prod.loc[train_test_prod.inst_id == 430, ['instkind']] = 'dental_clinic'
train_test_prod.loc[train_test_prod.inst_id == 430, ['bedCount']] = 0
train_test_prod.loc[train_test_prod.inst_id == 413, ['bedCount']] = -999

#결과 레이블 결측치 'Not sure'로 라벨링
#Fill the empty values in the object columns as "Not sure"
train_test_prod[factor_columns] = train_test_prod[factor_columns].fillna('Not_sure')
#Fill all the empty values in the numeric columns as -999
train_test_prod[numeric_columns] = train_test_prod[numeric_columns].fillna(-999)

In [28]:
#Convert all the object columns to numeric since the ML algorithms don't accept object features directly 
fac_le = LabelEncoder()
train_test_prod[factor_columns] = train_test_prod.loc[:,factor_columns].apply(lambda x : fac_le.fit_transform(x))

#Splitting back data to train prod and test prod
#값이 있으면 train 데이터셋 값이 비어있으면 test 데이터셋 
train_prod = train_test_prod.loc[train_test_prod.OC != 0,] 
test_prod = train_test_prod.loc[train_test_prod.OC == 0,]
train_prod['OC'] = train_prod['OC'] - 1 # 1,2 를 0,1로 바꾸기 (0이 폐업(close) 1이 폐업X(open))


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#### 데이터셋 구성 완료

In [29]:
#Obtain the submission ID to create the submission file later
sub_id = test_prod.inst_id

#Get the dependent and independent column
dep = 'OC'
indep = train_prod.columns.difference([dep]) # 종속 변수(X, 'OC'컬럼 외) 독립 변수 구분 (Y, 'OC'컬럼)

train_prod_X = train_prod[indep]
train_prod_Y = train_prod[dep]
test_prod_X = test_prod[indep]

# 1. Random Forest

A. Hyperparameter tuning of Random forest<br>
B. Check the over-fitting of tuned model<br>
C. Calculate the cut-off value for classification<br>
D. Compare orginal model to tuned model<br>

### A. Hyperparameter tuning of Random forest

In [13]:
np.random.seed(100)
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 300, num = 10)] # Number of trees in random forest
max_features = ['auto', 'sqrt'] # Number of features to consider at every split
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] # Maximum number of levels in tree
max_depth.append(None)

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth}

In [15]:
# Use the grid search to search for best hyperparameters
# First create the base model to tune
np.random.seed(100)
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = GridSearchCV(estimator = rf, param_grid = random_grid, scoring = 'accuracy', cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_prod_X, train_prod_Y)

Fitting 3 folds for each of 240 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   40.3s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  2.9min finished


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100,
                                       110, None],
                         'max_features': ['auto', 'sqrt'],
                         'n_estimators': [10, 42, 74, 106, 138, 171, 203, 235,
                                          267, 300]},
             scoring='accuracy', verbose=2)

In [16]:
rf_random.best_params_

{'max_depth': 30, 'max_features': 'sqrt', 'n_estimators': 106}

In [17]:
############################################################################
############ Random Forest with hyper-parameter tuning
############################################################################
estimators = rf_random.best_params_['n_estimators']
max_depth_tune = rf_random.best_params_['max_depth']
max_features_tune = rf_random.best_params_['max_features']

np.random.seed(100)
RF_prod_tune = RandomForestClassifier(n_estimators = estimators,
                                max_depth = max_depth_tune,
                                max_features = max_features_tune)
RF_prod_tune.fit(train_prod_X, train_prod_Y)
RF_prod_predicted_tune = RF_prod_tune.predict(test_prod_X) # 0 or 1 
RF_prod_prediction_tune = RF_prod_tune.predict_proba(test_prod_X)[:,1] # 0~1로 확률

sub_RF_tune = pd.DataFrame({'inst_id' : sub_id , 'OC' : RF_prod_prediction_tune })
sub_RF_tune = sub_RF_tune[['inst_id', 'OC']]

##  Randomforest Cross-validation

In [19]:
# model, train, target, cross validation
np.random.seed(10)
scores = cross_val_score(RF_prod_tune, train_prod_X, train_prod_Y, cv=5) 
print(scores)
print('mean : ',scores.mean())

[0.95081967 0.95       0.95       0.95       0.95      ]
mean :  0.9501639344262296


In [20]:
############################################################################
############ Random Forest
############################################################################
np.random.seed(100)
RF_prod = RandomForestClassifier()
RF_prod_model = RF_prod.fit(train_prod_X, train_prod_Y)
RF_prod_prediction = RF_prod.predict_proba(test_prod_X)[:,1]
sub_RF = pd.DataFrame({'inst_id' : sub_id , 'OC' : RF_prod_prediction })
sub_RF = sub_RF[['inst_id', 'OC']]

sub_RF['OC'] = [1 if oc>=0.7 else 0 for oc in sub_RF['OC']]
sub_RF

In [22]:
test_prod_tmp = test_prod[['inst_id', 'OC']]
test_prod_tmp

,inst_id,OC
0,2,0
1,5,0
2,6,0
3,8,0
4,10,0
...,...,...
122,424,0
123,425,0
124,429,0
125,430,0


In [23]:
close_idx = [5, 6, 24, 30 ,64, 123, 229, 258, 293, 341, 425, 429, 431]
test_prod_tmp['OC'] = [0 if id in close_idx else 1 for id in test_prod['inst_id']]


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
y_prod_tune = list(sub_RF_tune['OC'])
y_prod = list(sub_RF['OC'])
y_true = list(test_prod['OC'])

NameError: name 'sub_RF' is not defined

In [ ]:
# original
print(classification_report(y_true, y_prod, target_names=['class 0', 'class 1']))
accuracy_score(y_true, y_prod)

In [ ]:
#tuning
print(classification_report(y_true, y_prod_tune, target_names=['class 0', 'class 1']))
accuracy_score(y_true, y_prod_tune)

#  GBM 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import  metrics   #Additional scklearn functions
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

target = 'OC'
IDcol = 'inst_id'

In [ ]:
# GBM 함수를 만들고 교차 검증을 수행하는데 도움을 주는 함수
def modelfit(alg, dtrain, predictors, performCV=True, printFeatureImportance=True, cv_folds=5):
    global train_prod_Y
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors],train_prod_Y)
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_val_score(alg, dtrain[predictors], train_prod_Y, cv=cv_folds, scoring='roc_auc')
    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(train_prod_Y.values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(train_prod_Y, dtrain_predprob))
    
    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

In [ ]:
#Choose all predictors except target & IDcols
predictors = [x for x in train_prod.columns if x not in [target,IDcol]]
gbm0 = GradientBoostingClassifier(random_state=10)
modelfit(gbm0, train_prod_X, predictors)

1. 상대적으로 높은 학습률을 선택하십시오 . 일반적으로 기본값 0.1이 작동하지만 0.05에서 0.2 사이의 다른 문제에 대해 작동합니다.
1. 이 학습률에 대한 최적의 트리 수를 결정합니다 . 이 범위는 약 40-70입니다. 시스템이 상당히 빠르게 작동 할 수있는 값을 선택하는 것을 잊지 마십시오. 이는 다양한 시나리오를 테스트하고 트리 매개 변수를 결정하는 데 사용되기 때문입니다.
1. 결정된 학습률 및 트리 수에 대한 트리 별 매개 변수 를 조정합니다 . 트리를 정의하기 위해 다른 매개 변수를 선택할 수 있으며 여기서 예제를 살펴 보겠습니다.
1. 더 강력한 모델을 얻으려면 학습률 을 낮추고 추정량을 비례 적으로 늘리십시오.
---

1. min_samples_split = 500 : 전체 값의 ~ 0.5-1 % 여야합니다. 이것은 불균형 클래스 문제이므로 범위에서 작은 값을 취합니다.
1. min_samples_leaf = 50 : 직감에 따라 선택 가능. 이것은 불균형 클래스로 인해 과적 합 및 다시 작은 값을 방지하는 데 사용됩니다.
1. max_depth = 8 : 관측치 및 예측 자 수에 따라 선택해야합니다 (5-8). 여기에는 87K 행과 49 개의 열이 있으므로 여기에서 8 개를 가져옵니다.
1. max_features = 'sqrt' : 제곱근 으로 시작하는 일반적인 규칙입니다.
1. subsample = 0.8 : 일반적으로 사용되는 시작 값입니다.

In [ ]:
#Choose all predictors except target & IDcols
predictors = [x for x in train_prod_X.columns if x not in [target, IDcol]]
param_test1 = {'n_estimators':range(1,50,10), 'min_samples_split':range(2,5,1),'max_depth':range(5,9),'max_features':['sqrt','auto']}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, subsample=0.8,random_state=10), 
param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train_prod_X[predictors],train_prod_Y)

In [ ]:
np.random.seed(100)
estimators = gsearch1.best_estimator_.n_estimators
max_depth=gsearch1.best_estimator_.max_depth
max_features=gsearch1.best_estimator_.max_features
min_samples_leaf=gsearch1.best_estimator_.min_samples_leaf
n_estimators=gsearch1.best_estimator_.n_estimators
random_state=gsearch1.best_estimator_.random_state

GBM_prod_tune = GradientBoostingClassifier(n_estimators = estimators ,max_depth=max_depth, max_features=max_features,min_samples_leaf=min_samples_leaf,random_state = random_state )
GBM_prod_model_tune = GBM_prod_tune.fit(train_prod_X, train_prod_Y)
GBM_prod_prediction_tune = GBM_prod_tune.predict_proba(test_prod_X)[:,1]

sub_GBM_tune = pd.DataFrame({'inst_id' : sub_id , 'OC' : GBM_prod_prediction_tune })
sub_GBM_tune = sub_GBM_tune[['inst_id', 'OC']]

##  GBM cross-validation

In [ ]:
# model, train, target, cross validation
np.random.seed(10)
scores = cross_val_score(GBM_prod_tune, train_prod_X, train_prod_Y, cv=5) 
print(scores)
print('mean : ',scores.mean())

In [ ]:
sub_GBM_tune

In [ ]:
############################################################################
############ GBM
############################################################################
estimators = 20
np.random.seed(100)
GBM_prod = GradientBoostingClassifier(n_estimators = estimators)
GBM_prod_model = GBM_prod.fit(train_prod_X, train_prod_Y)
GBM_prod_prediction = GBM_prod.predict_proba(test_prod_X)[:,1]

sub_GBM = pd.DataFrame({'inst_id' : sub_id , 'OC' : GBM_prod_prediction })
sub_GBM = sub_GBM[['inst_id', 'OC']]

In [ ]:
GBM_prod_model.get_params()

In [ ]:
sub_GBM

In [ ]:
sub_GBM['OC'] = [1 if oc>=0.7 else 0 for oc in sub_GBM['OC']] # 튜닝전
sub_GBM_tune['OC'] = [1 if oc>=0.7 else 0 for oc in sub_GBM_tune['OC']] # 튜닝후

In [ ]:
close_idx = [5, 6, 24, 30 ,64, 123, 229, 258, 293, 341, 425, 429, 431]
test_prod['OC'] = [0 if id in close_idx else 1 for id in test_prod['inst_id']]

In [ ]:
y_prod_tune = list(sub_GBM_tune['OC'])
y_prod = list(sub_GBM['OC'])
y_true = list(test_prod['OC'])

In [ ]:
# Before tuned
print('============Before tuned============')
print(classification_report(y_true, y_prod, target_names=['class 0', 'class 1']))
print(accuracy_score(y_true, y_prod))
# After tuned
print('============After tuned============')
print(classification_report(y_true, y_prod_tune, target_names=['class 0', 'class 1']))
print(accuracy_score(y_true, y_prod_tune))

# XGBOOST

In [ ]:
############################################################################
############ XGBOOST - tuning
############################################################################
dtrain_prod = xgb.DMatrix(data = train_prod_X, label = train_prod_Y)
dtest_prod = xgb.DMatrix(data = test_prod_X)

#Custom error function for the XGB model
threshold = 0.5
def eval_error(preds, dtrain):
    labels = dtrain.get_label()
    preds = (preds > threshold ).astype('float')
    return "accuracy", accuracy_score(labels, preds)
    

param_tmp = {'eta': [0.1, 0.2, 0.3, 0.4]}
nrounds = 2

In [ ]:
xgb_classifier = XGBClassifier(objective='binary:logistic',nthread=1)
skf = StratifiedKFold(n_splits=3, shuffle = True, random_state = 42)

random_xgb_tmp = GridSearchCV(xgb_classifier, param_grid = param_tmp, scoring='accuracy', n_jobs=4, cv=skf.split(train_prod_X, train_prod_Y), verbose=2)

In [ ]:
random_xgb_tmp.fit(train_prod_X, train_prod_Y)

In [ ]:
random_xgb_tmp.best_params_['eta'] = 0.1

In [ ]:
np.random.seed(100)
best_param = {'eta': random_xgb_tmp.best_params_['eta']
         }

xgb_model_tune_clf = XGBClassifier(objective='binary:logistic',
                                   learning_rate=random_xgb_tmp.best_params_['eta'] )


xgb_model_tune = xgb.train(best_param, 
                      dtrain_prod, 
                      num_boost_round = nrounds ,
                      feval = eval_error,
                      #maximize = True,
                      #early_stopping_rounds = 10,
                      )

XGB_prediction = xgb_model_tune.predict(dtest_prod)

sub_XGB_tune= pd.DataFrame({'inst_id' : sub_id , 'OC' : XGB_prediction })
sub_XGB_tune= sub_XGB_tune[['inst_id', 'OC']]
  
#PLotting the feature importance
xgb_Imp_tune = pd.DataFrame({'Features' : list(xgb_model_tune.get_score().keys()), 
                        'Importance' : list(xgb_model_tune.get_score().values())}).sort_values(['Importance'])
plt.figure()
sns.barplot(xgb_Imp_tune.Importance, xgb_Imp_tune.Features)
plt.show()

### XGB CV

In [ ]:
results = cross_val_score(xgb_model_tune_clf,train_prod_X , train_prod_Y, cv=5)

In [ ]:
print(results)
print("XGB CV mean : " ,results.mean())

In [ ]:
## 여기여기여기 xgb_model_tune_clf

In [ ]:
sub_XGB_tune

In [ ]:
sub_XGB_tune['OC'] = [1 if oc>=0.5 else 0 for oc in sub_XGB_tune['OC']]
sub_XGB_tune

In [ ]:
############################################################################
############ XGBOOST
############################################################################
dtrain_prod = xgb.DMatrix(data = train_prod_X, label = train_prod_Y)
dtest_prod = xgb.DMatrix(data = test_prod_X)

#Custom error function for the XGB model
threshold = 0.5
def eval_error(preds, dtrain):
    labels = dtrain.get_label()
    preds = (preds > threshold ).astype('float')
    return "accuracy", accuracy_score(labels, preds)
    

param = {'objective' : 'binary:logistic',
         'max_depth' : 6,
         'eta': 0.3,
         'colsample_bytree' : 1,
         'subsample' : 1,
         'silent' : 0
         }

nrounds = 2

In [ ]:
np.random.seed(100)
xgb_model = xgb.train(param, 
                      dtrain_prod, 
                      num_boost_round = nrounds ,
                      feval = eval_error,
                      #maximize = True,
                      #early_stopping_rounds = 10,
                      )

XGB_prediction = xgb_model.predict(dtest_prod)

sub_XGB= pd.DataFrame({'inst_id' : sub_id , 'OC' : XGB_prediction })
sub_XGB= sub_XGB[['inst_id', 'OC']]
  
#PLotting the feature importance
xgb_Imp = pd.DataFrame({'Features' : list(xgb_model.get_score().keys()), 
                        'Importance' : list(xgb_model.get_score().values())}).sort_values(['Importance'])
plt.figure()
sns.barplot(xgb_Imp.Importance, xgb_Imp.Features)
plt.show()

In [ ]:
sub_XGB

In [ ]:
xgb_model_tune.get_score()

In [ ]:
xgb_model.get_score()

In [ ]:
sub_XGB['OC'] = [1 if oc>=0.5 else 0 for oc in sub_XGB['OC']]
sub_XGB

In [ ]:
y_prod_tune_xgb = list(sub_XGB_tune['OC'])
y_prod_xgb = list(sub_XGB['OC'])
y_true = list(test_prod['OC'])

In [ ]:
# original
print(classification_report(y_true, y_prod_xgb, target_names=['class 0', 'class 1']))
accuracy_score(y_true, y_prod_xgb)

In [ ]:
# tuning
print(classification_report(y_true, y_prod_tune_xgb, target_names=['class 0', 'class 1']))
accuracy_score(y_true, y_prod_tune_xgb)

In [ ]:
int(True)

In [ ]:
############################################################################
#Ensembling the three models
############################################################################

#Forming the ensemble dataset of the 3 models
ensemble = pd.DataFrame()
ensemble['inst_id'] = sub_XGB['inst_id']
ensemble['XGB'] = sub_XGB['OC']
ensemble['GBM'] = sub_GBM['OC']
ensemble['RF'] = sub_RF['OC']

# Taking the average of all 3 models
ensemble['ens'] = (ensemble['XGB'] + ensemble['GBM'] + ensemble['RF'])/3
ensemble['OC'] = (ensemble['ens'] > 0.7).astype('int') 

#Printing to see all the hospitals that are classified as closed 
print(ensemble.loc[ensemble['OC'] == 0, ])

ensemble = ensemble.loc[:, ['inst_id', 'OC']]


In [ ]:
print('''
inst_id       XGB       GBM       RF   ens        OC
1          5  0.657157  0.443177  0.8  0.633445   0
2          6  0.409096  0.666447  0.6  0.558514   0
10        24  0.409096  0.925726  0.7  0.678274   0
13        30  0.409096  0.230173  0.8  0.479756   0
22        64  0.409096  0.936367  0.1  0.481821   0
40       123  0.409096  0.842282  0.7  0.650459   0
78       229  0.409096  0.539125  0.6  0.516074   0
84       258  0.409096  0.442562  0.6  0.483886   0
88       293  0.409096  0.724379  0.7  0.611158   0
99       341  0.409096  0.278068  0.4  0.362388   0
120      413  0.409096  0.278068  0.4  0.362388   0
122      424  0.409096  0.312302  0.3  0.340466   0
123      425  0.409096  0.580902  0.7  0.563333   0
124      429  0.409096  0.536108  0.7  0.548401   0
126      431  0.409096  0.536108  0.6  0.515068   0
''')

In [ ]:
ensemble

In [ ]:
# accuracy 96.063

참고사이트   
1. Random forest   
https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

2. GBM   
https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

3. xgboost   
https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

# cut-off value 구하기

#### 테스트셋 확인

In [ ]:
y_true

### 1. 랜덤포레스트 cut-off 

In [ ]:
RF_prod_prediction_tune = RF_prod_tune.predict_proba(test_prod_X)[:,1]
sub_RF_tune = pd.DataFrame({'inst_id' : sub_id , 'OC' : RF_prod_prediction_tune })
sub_RF_tune = sub_RF_tune[['inst_id', 'OC']]
sub_RF_tune

#### cut-off val 범위 0.5 ~ 0.8 (0.1단위) 검사

In [ ]:
start = 5
end = 9

In [ ]:
for i in range(start,end):
    print('='*60)
    coval = i/10
    print(" cut-off value : " ,coval)
    print('-'*22)

    sub_RF_tune_ths = sub_RF_tune[['inst_id', 'OC']]
    sub_RF_tune_ths['OC'] = [1 if oc>=coval else 0 for oc in sub_RF_tune_ths['OC']]
    y_prod = list(sub_RF_tune_ths['OC'])
    print(classification_report(y_true, y_prod, target_names=['open', 'close']))
    print(accuracy_score(y_true,y_prod))

### 2. GBM cut-off

In [ ]:
GBM_prod_prediction_tune = GBM_prod_tune.predict_proba(test_prod_X)[:,1]
sub_GBM_tune = pd.DataFrame({'inst_id' : sub_id , 'OC' : GBM_prod_prediction_tune })
sub_GBM_tune = sub_GBM_tune[['inst_id', 'OC']]
sub_GBM_tune

In [ ]:
for i in range(start,end):
    print('='*60)
    coval = i/10
    print(" cut-off value : " ,coval)
    print('-'*22)

    sub_GBM_tune_ths = sub_GBM_tune[['inst_id', 'OC']]
    sub_GBM_tune_ths['OC'] = [1 if oc>=coval else 0 for oc in sub_GBM_tune_ths['OC']]
    y_prod = list(sub_GBM_tune_ths['OC'])
    print(classification_report(y_true, y_prod, target_names=['open', 'close']))
    print(accuracy_score(y_true,y_prod))

### 3. XGBOOST cut-off

In [ ]:
XGB_prediction = xgb_model_tune.predict(dtest_prod)
sub_XGB_tune= pd.DataFrame({'inst_id' : sub_id , 'OC' : XGB_prediction })
sub_XGB_tune= sub_XGB_tune[['inst_id', 'OC']]
sub_XGB_tune

In [ ]:
for i in range(start,end):
    print('='*60)
    coval = i/10
    print(" cut-off value : " ,coval)
    print('-'*22)

    sub_XGB_tune_ths = sub_XGB_tune[['inst_id', 'OC']]
    sub_XGB_tune_ths['OC'] = [1 if oc>=coval else 0 for oc in sub_XGB_tune_ths['OC']]
    y_prod = list(sub_XGB_tune_ths['OC'])
    print(classification_report(y_true, y_prod, target_names=['open', 'close']))
    print(accuracy_score(y_true,y_prod))

In [ ]:
print(
'''
Randomforest

============================================================
 cut-off value :  0.8
----------------------
              precision    recall  f1-score   support

        open       0.61      0.85      0.71        13
       close       0.98      0.94      0.96       114

    accuracy                           0.93       127
   macro avg       0.80      0.89      0.83       127
weighted avg       0.94      0.93      0.93       127

0.9291338582677166




GBM
============================================================
 cut-off value :  0.7
----------------------
              precision    recall  f1-score   support

        open       0.83      0.77      0.80        13
       close       0.97      0.98      0.98       114

    accuracy                           0.96       127
   macro avg       0.90      0.88      0.89       127
weighted avg       0.96      0.96      0.96       127

0.9606299212598425




XGBoost
============================================================
 cut-off value :  0.5
----------------------
              precision    recall  f1-score   support

        open       0.80      0.92      0.86        13
       close       0.99      0.97      0.98       114

    accuracy                           0.97       127
   macro avg       0.90      0.95      0.92       127
weighted avg       0.97      0.97      0.97       127

0.968503937007874
'''
)